In [1]:
!pip install deal 
!pip install openpyxl

  Obtaining dependency information for deal from https://files.pythonhosted.org/packages/9b/f6/ec36cd2cf124d402236db9bff90cc535d85eea44e5e7dc602a5887d24275/deal-4.24.3-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.9/206.9 kB 312.8 kB/s eta 0:00:0000:0100:01


In [1]:
from deap import base, creator, tools, algorithms
import pandas as pd
import numpy as np
import random

employees_df = pd.read_excel('/home/wissal/Téléchargements/Updated_Employees_Data.xlsx')  
tasks_df = pd.read_excel('/home/wissal/Téléchargements/New_Tasks_Data.xlsx')  

employees_data = {
    row['Employee_ID']: {
        'days': list(map(int, row['Available_Days'].split(', '))),
        'shifts': list(map(int, row['Available_Shifts'].split(', '))),
        'skills': row['Skills'].split(', ')
    } for _, row in employees_df.iterrows()
}

tasks_data = {
    row['Task_ID']: {
        'required_skill': row['Required_Skill'],
        'assembly_line': row['Assembly_Line']
    } for _, row in tasks_df.iterrows()
}

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("attr_emp", random.choice, list(employees_data.keys()))
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_emp, len(tasks_data))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evaluate(individual):
    score = 0
    for i, task_id in enumerate(tasks_data):
        emp_id = individual[i]
        task = tasks_data[task_id]
        employee = employees_data[emp_id]

        # Check if employee has required skill and is available
        if task['required_skill'] in employee['skills']:
            score += 1

    return score,

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

def run_ga():
    pop = toolbox.population(n=100)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)

    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=50, 
                                   stats=stats, halloffame=hof, verbose=True)
    
    return pop, log, hof

population, logbook, hof = run_ga()

best_solution = hof[0]
best_solution_fitness = best_solution.fitness.values[0]

assignment = {task_id: best_solution[i] for i, task_id in enumerate(tasks_data)}

assignment_df = pd.DataFrame(list(assignment.items()), columns=['Task_ID', 'Assigned_Employee_ID'])

assignment_df


gen	nevals	avg  	min	max
0  	100   	18.37	10 	25 
1  	60    	20.99	15 	25 
2  	57    	22.8 	20 	27 
3  	70    	24.13	18 	27 
4  	53    	25.28	20 	27 
5  	55    	26.33	23 	29 
6  	64    	26.91	24 	29 
7  	48    	27.58	25 	29 
8  	63    	28.06	24 	30 
9  	62    	28.67	26 	30 
10 	54    	29.14	27 	30 
11 	51    	29.29	26 	30 
12 	53    	29.78	27 	30 
13 	63    	29.81	27 	30 
14 	59    	29.83	27 	30 
15 	70    	29.86	27 	30 
16 	65    	29.84	26 	30 
17 	62    	29.91	27 	30 
18 	58    	29.78	27 	30 
19 	62    	29.7 	26 	30 
20 	64    	29.79	27 	30 
21 	46    	29.81	26 	30 
22 	70    	29.82	27 	30 
23 	62    	29.81	27 	30 
24 	65    	29.87	27 	30 
25 	66    	29.82	26 	30 
26 	60    	29.85	28 	30 
27 	53    	29.84	26 	30 
28 	65    	29.85	26 	30 
29 	49    	29.85	27 	30 
30 	59    	29.87	27 	30 
31 	57    	29.83	26 	30 
32 	65    	29.77	27 	30 
33 	66    	29.84	28 	30 
34 	58    	29.85	28 	30 
35 	64    	29.84	27 	30 
36 	67    	29.9 	28 	30 
37 	60    	29.82	28 	30 
38 	60    	29.75	26 	30 


,Task_ID,Assigned_Employee_ID
0,T_001,E_046
1,T_002,E_029
2,T_003,E_004
3,T_004,E_015
4,T_005,E_018
5,T_006,E_050
6,T_007,E_044
7,T_008,E_040
8,T_009,E_037
9,T_010,E_008
